In [ ]:
# 📚 Imports (if not already present)
import pandas as pd
import yfinance as yf
import numpy as np
import joblib
import requests
import os
from datetime import datetime
from ta.momentum import RSIIndicator
from ta.trend import MACD, ADXIndicator

# 🔐 Load Telegram & Model
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID  = os.getenv("TELEGRAM_CHAT_ID")
model             = joblib.load("fibonacci_xgb_model.pkl")

# 🗃️ Load the full F&O stock list
df_symbols    = pd.read_csv("futures_list.csv")
futures_list  = df_symbols["Symbol"].dropna().tolist()
print(f"Loaded {len(futures_list)} F&O symbols.")

# 🔔 Telegram helper
def send_telegram(msg, chart_path=None):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto" if chart_path else f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    data = {'chat_id': TELEGRAM_CHAT_ID, 'caption': msg} if chart_path else {'chat_id': TELEGRAM_CHAT_ID, 'text': msg}
    files = {'photo': open(chart_path, 'rb')} if chart_path else None
    resp = requests.post(url, data=data, files=files)
    print("Telegram response:", resp.status_code)

# 🔁 Main loop over every symbol
today = datetime.today().strftime("%Y-%m-%d")
if not os.path.exists("sent_signals.csv"):
    pd.DataFrame(columns=["date","symbol"]).to_csv("sent_signals.csv", index=False)
sent = pd.read_csv("sent_signals.csv")

results = []
for symbol in futures_list:
    try:
        df = yf.download(f"{symbol}.NS", period="6mo", interval="1d", progress=False)
        if df.empty:
            continue

        # 1) Indicators
        df["RSI"] = RSIIndicator(df["Close"]).rsi()
        macd      = MACD(df["Close"])
        df["MACD_Hist"] = macd.macd_diff()
        df["ADX"] = ADXIndicator(df["High"], df["Low"], df["Close"]).adx()

        # 2) Fibonacci 0.618 retracement
        high = df["High"].rolling(10).max()
        low  = df["Low"].rolling(10).min()
        retracement = high - (high - low) * 0.618
        df["near_618"] = df["Low"] <= retracement

        # 3) ATR-based SL/TP (or fixed fallback)
        # ... (your existing ATR logic here) ...

        # 4) LSTM trend exhaustion filter
        # ... (your LSTM placeholder call here) ...

        latest = df.iloc[-1]
        if latest["near_618"] and latest["RSI"] > 50 and latest["MACD_Hist"] > 0:
            # XGBoost confidence
            feats      = latest[["RSI","MACD_Hist","ADX","Volume"]].fillna(0).values.reshape(1,-1)
            confidence = model.predict_proba(feats)[0][1]

            # Dedupe and threshold
            if confidence >= 0.70 and not ((sent["symbol"]==symbol)&(sent["date"]==today)).any():
                # Generate and save chart to 'chart.png'
                # ... (your matplotlib + fib + candle + volume code) ...

                msg = (
                    f"🚨 {symbol} Reversal Alert\n"
                    f"Date: {today}\n"
                    f"Conf: {confidence:.1%}\n"
                    f"RSI: {latest['RSI']:.1f}, MACD: {latest['MACD_Hist']:.2f}\n"
                    f"SL: {sl_price}, TP: {tp_price}"
                )
                send_telegram(msg, chart_path="chart.png")

                # Log it
                sent = sent.append({"date":today,"symbol":symbol}, ignore_index=True)
                results.append({"Symbol":symbol,"Conf":confidence})
                sent.to_csv("sent_signals.csv", index=False)
    except Exception as e:
        print(f"Error {symbol}: {e}")

# 📊 Summary
pd.DataFrame(results)
